In [ ]:
import pandas as pd
import numpy as np
import uuid

june_july_dataset_path = 'data/June and July dataset/'
jan_feb_dataset_path = 'data/January_February_dataset/'

In [ ]:
# Load the data
viewers_by_fetch_part1 = pd.read_json(june_july_dataset_path + 'Viewers_by_fetch_part1.json', lines=True, dtype={'video': 'object', 'viewers_count': 'int32'})
viewers_by_fetch_part2 = pd.read_json(june_july_dataset_path + 'viewers_by_fetch_part2.json', lines=True, dtype={'video': 'object', 'viewers_count': 'int32'})
jan_viewers_by_fetch = pd.read_json(jan_feb_dataset_path + 'viewers_by_fetch_period1and2.json', lines=True, dtype={'video': 'object', 'viewers_count': 'int32'})
viewers_by_fetch = pd.concat([viewers_by_fetch_part1, viewers_by_fetch_part2, jan_viewers_by_fetch], ignore_index=True)
del viewers_by_fetch_part1, viewers_by_fetch_part2, jan_viewers_by_fetch

# Convert 'video' column
viewers_by_fetch["video"] = viewers_by_fetch["video"].apply(lambda x: x['$oid'] if isinstance(x, dict) else x)

# Convert 'refresh_time' column to datetime
viewers_by_fetch["refresh_time"] = pd.to_datetime(viewers_by_fetch["refresh_time"])

# Optimize memory usage
viewers_by_fetch['viewers_count'] = pd.to_numeric(viewers_by_fetch['viewers_count'], downcast='unsigned')
viewers_by_fetch['_id'] = viewers_by_fetch['_id'].apply(lambda x: x['$oid'] if isinstance(x, dict) else x)
viewers_by_fetch['_id'] = viewers_by_fetch['_id'].astype('category')
viewers_by_fetch['video'] = viewers_by_fetch['video'].astype('category')

display(viewers_by_fetch.info(memory_usage='deep'))

In [5]:
viewers_threshold = 1000
max_viewers = viewers_by_fetch.groupby('video')['viewers_count'].max()
videos_with_high_viewers = max_viewers[max_viewers >= viewers_threshold].index
videos_with_high_viewers = viewers_by_fetch[viewers_by_fetch['video'].isin(videos_with_high_viewers)]
videos_with_high_viewers = videos_with_high_viewers.sort_values(by=['video', 'refresh_time'])

C:\Users\admin2\AppData\Local\Temp\ipykernel_33832\3749116294.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  max_viewers = viewers_by_fetch.groupby('video')['viewers_count'].max()


In [16]:
# get one video of videos_with_high_viewers
video = videos_with_high_viewers['video'].iloc[0]
video_viewers = videos_with_high_viewers[videos_with_high_viewers['video'] == video][['refresh_time', 'viewers_count']]
video_viewers = video_viewers.set_index('refresh_time')

In [ ]:
user_events = {
    "event": [],
    "id": []
}

def generate_uuid():
    return str(uuid.uuid5())

publisher_id = None
viewers_ids = []
for i in range(0, len(video_viewers)):
    if i == 0:
        user_events["event"].append(0)
        publisher_id = generate_uuid()
        user_events["id"].append(publisher_id)
    elif i == len(video_viewers) - 1:
        user_events["event"].append(2)
        user_events["id"].append(publisher_id)
    else:
        user_events["event"].append(1)
        user_events["id"].append(video_viewers.index[i])